In [3]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [6]:
lemmatizer = WordNetLemmatizer
dev_light = json.loads(open('dev_light.json').read())

words = []
classes = []
documents = []
